In [6]:
import pandas as pd
import sqlalchemy
import mysql.connector

#Pour que python reconnaisse un dossier comme un module il faut ajouter en son sein un fichier __init__.py 
#et également l'ajouter au PATH

import sys  

sys.path.insert(0, "/home/apprenant/Desktop/AmericanDream/")


#On peut ensuite importer le dossier comme un module

from src.d00_utils.mysql_utils import mysql_connect, save_to_mysql
connect = mysql_connect()

Importation des données depuis my sql


In [ ]:
data = pd.read_sql("Analyst",con=connect)
data2 = pd.read_sql("Salary",con=connect)

##1. Selection des colonnes
J'affiche un échantillno de mes données pour analyser la pertinence des différentes colonnes    

In [10]:
print(data2.head())
print(data2.columns)

   index  Survey Year           Timestamp  SalaryUSD        Country  \
0      0         2020 2020-01-04 18:50:34   115000.0  United States   
1      1         2020 2020-01-04 10:43:02   100000.0  United States   
2      2         2020 2020-01-04 09:51:46   100000.0          Spain   
3      3         2020 2020-01-04 01:08:54    70000.0  United States   
4      4         2020 2020-01-03 15:28:54   110000.0  United States   

  PostalCode       PrimaryDatabase  YearsWithThisDatabase  \
0         03  Microsoft SQL Server                     15   
1       None                 Other                      6   
2      28046  Microsoft SQL Server                      2   
3      94133  Microsoft SQL Server                      3   
4      95354                Oracle                     30   

                                      OtherDatabases    EmploymentStatus  ...  \
0  Microsoft SQL Server, MongoDB, Azure SQL DB (a...  Full time employee  ...   
1                                      MySQL

In [11]:
data2 = data2[['index','SalaryUSD','Country','PostalCode','EmploymentStatus','JobTitle','ManageStaff','YearsWithThisTypeOfJob',
          'HowManyCompanies','OtherPeopleOnYourTeam', 'CompanyEmployeesOverall',
          'PopulationOfLargestCityWithin20Miles','EmploymentSector', 'LookingForAnotherJob', 'CareerPlansThisYear',
          'Gender', 'OtherJobDuties']]

In [13]:
print(data2.isnull().sum()) 
print(data2.shape)

index                                      0
SalaryUSD                                  0
Country                                    0
PostalCode                              1390
EmploymentStatus                           0
JobTitle                                   0
ManageStaff                                0
YearsWithThisTypeOfJob                     0
HowManyCompanies                           0
OtherPeopleOnYourTeam                      0
CompanyEmployeesOverall                    0
PopulationOfLargestCityWithin20Miles       0
EmploymentSector                           0
LookingForAnotherJob                       0
CareerPlansThisYear                        0
Gender                                     0
OtherJobDuties                           620
dtype: int64
(8627, 17)
